<h1 align=center><font size = 5>Capstone Project -The Battle of Neighborhoods </font>
<br><font size = 3>Murphy Chan Ngar Kay</font>
<br><font size = 3>October 22, 2020</font>
</h1>

## Introduction and Background
<br>
The concept of neighborhoods used by the City of Toronto were developed to help
government and community organizations with their local planning by providing socioeconomic data at a meaningful geographic area. The boundaries of these social
planning neighborhoods do not change over time, allowing researchers to examine
changes over time.<br>
This project is going to illustrate how data analysis helps to be creative and come up with
different ideas to solve problem. For example, compare different neighborhoods in terms
of demographic data. Building a profile for each neighborhood for potential explanation
of why a neighborhood is popular. The cause of complaints in another neighborhood, or
anything else related to neighborhoods. So, the name of the capstone project is the battle
of the neighborhoods.<br>
Several datasets from different sources will be considered for leverage location data to
solve a problem or to get deeper insights into a neighborhood's reputation. To create the
different view of analysis, the Foursquare API will be used for the location data to
explore or compare neighborhoods or cities, in particular this project is focus on Canada.<br>


## Problem Description

Consider Hiroshima Company is going to startup a business of opening a sushi restaurant
in Toronto which targeting middle class customers with location in :Etobicoke or
Scarborough. On top of the food quality, service and any other interior elements of the
restaurant, one of the most important thing is the location and neighborhoods of the
restaurant location. This project is to illustrate how data impacts on the determination for
problem solving. The following analysis mainly compares the 2 boroughs in
Toronto :Etobicoke and Scarborough <br>

## Description of Data 
<br>
In the following data analysis, will try to illustrate the decision-making criteria supported by the data source. For data reference, the website, wiki, will be used for Postal code, Borough, and Neighborhood link :
<a href = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"> https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M </a>
    
 The location data for analyzing further information for the neighborhoods with latitude and longitude coordinates are from the following link :
http://cocl.us/Geospatial_data <br>
For the geographic data, Latitude and Longitude will be used for data mapping. While the Foursquare API will be called for the information of nearby venues and the most common venue to analysis the popularity of special location.

Data for the demographic statistic figures will be used for the comparison and scoring for each location. The analysis is based on the profile of different neighborhoods, including the Population, Income, and age distribution. The data source is from :
https://open.toronto.ca/
<br>


## Import necessary Libraries

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import json # library to handle JSON files

from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
from bs4 import BeautifulSoup
import geocoder
import folium 

print('Libraries imported.')

Libraries imported.


Loading Toronto dataset

In [3]:
source_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source_file = requests.get(source_url).text
bsoup = BeautifulSoup(source_file, 'html.parser')
Toronto =bsoup.find('table')
df_header = ['Postalcode','Borough','Neighborhood']
Toronto_tr = Toronto.tbody.find_all("tr")
temp = []
for tr in Toronto_tr:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned\n":
        for i, trow in enumerate(row):
            str = trow
            row[i] = str.replace("\n","")
        temp.append(row)
        
df = pd.DataFrame(temp, columns = df_header)
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
df_tmp=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df_tmp

Postalcode
M1B                                       Malvern, Rouge
M1C               Rouge Hill, Port Union, Highland Creek
M1E                    Guildwood, Morningside, West Hill
M1G                                               Woburn
M1H                                            Cedarbrae
                             ...                        
M9N                                               Weston
M9P                                            Westmount
M9R    Kingsview Village, St. Phillips, Martin Grove ...
M9V    South Steeles, Silverstone, Humbergate, Jamest...
M9W                  Northwest, West Humber - Clairville
Name: Neighborhood, Length: 103, dtype: object

In [5]:
df_tmp=df_tmp.reset_index(drop=False)
df_tmp.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)
df_merge = pd.merge(df, df_tmp, on='Postalcode')
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

df_merge.drop_duplicates(inplace=True)
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
df_merge.sort_values(by=['Postalcode'])
df_merge.head()


,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
print("The shape of dataframe : ",df_merge.shape)

The shape of dataframe :  (103, 3)


<br>
Use the Geocoder package or the csv file to create the required dataframe.

In [7]:
df_geo_coor = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude
df_geo_coor.rename(columns={'Postal Code':'Postalcode'},inplace=True)

# Merge the 2 tables into a new one with key Postalcode
geo_merged = pd.merge(df_merge,df_geo_coor, on='Postalcode')
geo_merged

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [22]:
geo_merged = pd.read_csv("geo_merged.csv")
# select data which contains the key word Toronto
Scarborough_df=geo_merged[geo_merged['Borough'].str.contains("Scarborough")]
Etobicoke_df =  geo_merged[geo_merged['Borough'].str.contains("Etobicoke")]

In [30]:
# set Foursquare parameters 
CLIENT_ID = ' ' # remove private information
CLIENT_SECRET = ' ' # remove private information
VERSION = '20180604'

# define function for getting near by venuses 
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # pass the parameter to Foursquare API 
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
# Call the defined function by passing the list of Toronto_df
Scarborough_df_venues = getNearbyVenues(names=Scarborough_df['Neighborhood'],
                                   latitudes=Scarborough_df['Latitude'],
                                   longitudes=Scarborough_df['Longitude']
                                  )
Etobicoke_df_venues = getNearbyVenues(names=Etobicoke_df['Neighborhood'],
                                   latitudes=Etobicoke_df['Latitude'],
                                   longitudes=Etobicoke_df['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Parkview Hill, Woodbine Gardens
Glencairn
Don Mills
Woodbine Heights
Humewood-Cedarvale
Caledonia-Fairbanks
Leaside
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Del Ray, Mount Dennis, Keelsdale and Silverthorn
Humberlea, Emery
Willowdale, Willowdale East
Downsview
Runnymede, The Junction North
Weston
York Mills West
Willowdale, Willowdale West


In [8]:
Scarborough_df_venues.head()

,Unnamed: 0,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [9]:
Etobicoke_df_venues.groupby('Neighborhood').count()

,Unnamed: 0,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
"Alderwood, Long Branch",7,7,7,7,7,7,7
"Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood",8,8,8,8,8,8,8
"Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens",4,4,4,4,4,4,4
"Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West",16,16,16,16,16,16,16
"New Toronto, Mimico South, Humber Bay Shores",14,14,14,14,14,14,14
"Northwest, West Humber - Clairville",4,4,4,4,4,4,4
"Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East",2,2,2,2,2,2,2
"South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens",10,10,10,10,10,10,10
"The Kingsway, Montgomery Road, Old Mill North",2,2,2,2,2,2,2


In [11]:
print('The distinct value of Etobicoke Venue Category : {}'.format(len(Etobicoke_df_venues['Venue Category'].unique())))
print('The distinct value of Scarborough Venue Category : {}'.format(len(Scarborough_df_venues['Venue Category'].unique())))

The distinct value of Etobicoke Venue Category : 41
The distinct value of Scarborough Venue Category : 55


In [59]:

# get one dummy from result_venues
Scarborough_onehot = pd.get_dummies(Scarborough_df_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scarborough_onehot['Neighborhood'] = Scarborough_df_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Scarborough_onehot.columns[-1]] + list(Scarborough_onehot.columns[:-1])
Scarborough_onehot = Scarborough_onehot[fixed_columns]
Scarborough_by_Neighborhood = Scarborough_onehot.groupby('Neighborhood').mean().reset_index()

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)    
    return row_categories_sorted.index.values[0:num_top_venues]

# check for the top 10 common venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Scarborough_by_Neighborhood['Neighborhood']

for ind in np.arange(Scarborough_by_Neighborhood.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scarborough_by_Neighborhood.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,Gas Station,Fried Chicken Joint
2,Cedarbrae,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Gym
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Chinese Restaurant,Noodle House,Thai Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Intersection,Bank,Italian Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",Motel,American Restaurant,Vietnamese Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [60]:
Scarborough_venues_sorted = neighborhoods_venues_sorted

In [7]:
# to get the 5 clustering
kclusters = 5

Scarborough_clustering = Scarborough_by_Neighborhood.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Scarborough_clustering)
kmeans.labels_[0:20] 

NameError: name 'Scarborough_by_Neighborhood' is not defined

In [62]:
# add clustering labels
Scarborough_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Scarborough_merged = Scarborough_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Scarborough_merged = Scarborough_merged.join(Scarborough_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Scarborough_merged.head()

,Unnamed: 0,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2.0,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Department Store
12,12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,4.0,Bar,Construction & Landscaping,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
18,18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Rental Car Location,Electronics Store,Medical Center,Intersection,Bank,Restaurant,Mexican Restaurant,Breakfast Spot,College Stadium,Construction & Landscaping
22,22,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Mexican Restaurant,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
26,26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Gym


In [10]:
Scarborough_merged = pd.read_csv("Scarborough_merged.csv")

In [8]:
address = 'Etobicoke, ON'
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

Etobicoke_merged_nonan = Etobicoke_merged.dropna(subset=['Cluster Labels'])

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Etobicoke_merged['Latitude'], Etobicoke_merged['Longitude'], Etobicoke_merged['Neighborhood'], Etobicoke_merged['Cluster Labels']):
    label = '{} Cluster {}'.format(poi, cluster)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

In [11]:
address = 'Scarborough, ON'
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

Scarborough_merged_nonan = Scarborough_merged.dropna(subset=['Cluster Labels'])
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Scarborough_merged['Latitude'], Scarborough_merged['Longitude'], Scarborough_merged['Neighborhood'], Scarborough_merged['Cluster Labels']):
    label = '{} Cluster {}'.format(poi, cluster)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster -1],
        fill_opacity=0.7).add_to(map_clusters)
    
    
map_clusters